In [1]:
import os, sys
import gc
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
sys.path.append('../')
import torch

In [3]:
from get_algos import get_all_algos , run_experiment ,create_paths, get_action_type_name

In [4]:
import gym
from sb3_contrib import QRDQN ,RecurrentPPO,TRPO 
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.env_util import make_atari_env

### 1. Explore breakout environment
<ul>
    <li>Explore the environment</li>
</ul>

In [18]:
env_name='Breakout-v0'
env=gym.make(env_name)

In [19]:
print('Action space: ',env.action_space)
print('Observation space shape: ',env.observation_space)
print('Action space shape: ',env.action_space.shape)
print('Observation space shape: ',env.observation_space.shape)

Action space:  Discrete(4)
Observation space shape:  Box([[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]], [[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [

In [20]:
#Taking a look at how breakout looks and simulate random actions
env.reset()
rewards_all=[]
for epi in range(50):
    state=env.reset()
    rewards=0
    while True:
        action=env.action_space.sample()
        state,reward,done,_=env.step(action)
        env.render()
        rewards+=reward
        if done:
            rewards_all.append(rewards)
            if (epi+1)%5==0:
                print('Episode: ',epi+1,' Reward: ',rewards)
            break  
env.close()
print('Average rewards for random action Breakout-v0 is: ',sum(rewards_all)/len(rewards_all))

E:\Anaconda\envs\ai_gym\lib\site-packages\gym\envs\atari\environment.py:269: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  "We strongly suggest supplying `render_mode` when "


Episode:  5  Reward:  4.0
Episode:  10  Reward:  1.0
Episode:  15  Reward:  1.0
Episode:  20  Reward:  1.0
Episode:  25  Reward:  5.0
Episode:  30  Reward:  1.0
Episode:  35  Reward:  0.0
Episode:  40  Reward:  1.0
Episode:  45  Reward:  0.0
Episode:  50  Reward:  1.0
Average rewards for random action Breakout-v0 is:  1.36


### 2. Training
<ol>
    <li>Make the atari env using make_atari_env</li>
    <li>Parallelize the learning using VecFrameStack</li>
    <li>Create train function for all appropriate algos</li>
    <li>Train</li>
    <li>Store model</li>
</ol> 

In [8]:
env_recurrent_type={'Discrete':['MaskablePPO', 'QRDQN', 'RecurrentPPO', 'TRPO'],'Box':['ARS', 'RecurrentPPO', 'TQC', 'TRPO'], 
                    'MultiDiscrete':['MaskablePPO', 'RecurrentPPO', 'TRPO'],'MultiBinary':['MaskablePPO', 'RecurrentPPO', 'TRPO']}
def get_algo_recurrent(env):
    act_space=str(type(env.action_space)).split('.')[3].split("'")[0]
    return env_recurrent_type[act_space]
    

In [9]:
algo_list=get_algo_recurrent(gym.make(env_name))
print(algo_list)
if 'MaskablePPO' in algo_list:
    algo_list.remove('MaskablePPO')

In [11]:
def train_breakout(algo_name, env_name,n_steps,device='cuda'):
    if algo_name =='QRDQN' or 'TRPO':
        device='cpu'
    else:
        device='cuda' #GPU was unable to load the entire experience relay so moved to virtual memory on disk
    policy='CnnLstmPolicy' if algo_name=='RecurrentPPO' else 'CnnPolicy'        
    log_path, render_path, model_path=create_paths(algo_name,env_name,n_steps)
    env=make_atari_env(env_name,n_envs=6,seed=0)
    env=VecFrameStack(env, n_stack=6)
    model=eval(algo_name)(policy,env, tensorboard_log=log_path,verbose=1,device=device)
    model.learn(n_steps,)
    model.save(os.path.join(model_path,env_name+"_"+algo_name+"_"+"model"))
    del model
    env.close()
    gc.collect()
    torch.cuda.empty_cache()

    return log_path, render_path, model_path

In [12]:
models_records={}
n_steps=200000
env_name='Breakout-v0'

In [14]:
def train_experiment(algo_list, env_name,n_steps, models_records):
    for algo in reversed(algo_list):
        model_record={}
        train_breakout(algo, env_name,n_steps)

In [15]:
train_experiment(algo_list, env_name, 200000, models_records)

Using cpu device
Wrapping the env in a VecTransposeImage.


E:\Anaconda\envs\ai_gym\lib\site-packages\stable_baselines3\common\buffers.py:229: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 84.69GB > 12.96GB
  "This system does not have apparently enough memory to store the complete "


Logging to Breakout-v0\QRDQN_200000\logs\QRDQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.958    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 151      |
|    time_elapsed     | 0        |
|    total_timesteps  | 42       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.923    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 212      |
|    time_elapsed     | 0        |
|    total_timesteps  | 78       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.881    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 257      |
|    time_elapsed     | 0        |
|    total_timesteps  | 120      |
----------------------------------
------

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 266      |
|    ep_rew_mean      | 1.13     |
|    exploration_rate | 0.0734   |
| time/               |          |
|    episodes         | 92       |
|    fps              | 353      |
|    time_elapsed     | 2        |
|    total_timesteps  | 936      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 261      |
|    ep_rew_mean      | 1.06     |
|    exploration_rate | 0.0377   |
| time/               |          |
|    episodes         | 96       |
|    fps              | 353      |
|    time_elapsed     | 2        |
|    total_timesteps  | 972      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 262      |
|    ep_rew_mean      | 1.06     |
|    exploration_rate | 0.01     |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 268      |
|    ep_rew_mean      | 1.15     |
|    exploration_rate | 0.01     |
| time/               |          |
|    episodes         | 180      |
|    fps              | 356      |
|    time_elapsed     | 5        |
|    total_timesteps  | 1824     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 270      |
|    ep_rew_mean      | 1.2      |
|    exploration_rate | 0.01     |
| time/               |          |
|    episodes         | 184      |
|    fps              | 356      |
|    time_elapsed     | 5        |
|    total_timesteps  | 1860     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 269      |
|    ep_rew_mean      | 1.19     |
|    exploration_rate | 0.01     |
| time/               |          |
|    episodes       

...

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 297      |
|    ep_rew_mean      | 1.98     |
|    exploration_rate | 0.01     |
| time/               |          |
|    episodes         | 17996    |
|    fps              | 148      |
|    time_elapsed     | 1326     |
|    total_timesteps  | 196584   |
| train/              |          |
|    learning_rate    | 5e-05    |
|    loss             | 3.68     |
|    n_updates        | 6107     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 297      |
|    ep_rew_mean      | 1.98     |
|    exploration_rate | 0.01     |
| time/               |          |
|    episodes         | 18000    |
|    fps              | 148      |
|    time_elapsed     | 1326     |
|    total_timesteps  | 196620   |
| train/              |          |
|    learning_rate    | 5e-05    |
|    loss             | 1.83     |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 301      |
|    ep_rew_mean      | 2.03     |
|    exploration_rate | 0.01     |
| time/               |          |
|    episodes         | 18060    |
|    fps              | 148      |
|    time_elapsed     | 1332     |
|    total_timesteps  | 197268   |
| train/              |          |
|    learning_rate    | 5e-05    |
|    loss             | 3.5      |
|    n_updates        | 6136     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 301      |
|    ep_rew_mean      | 2.03     |
|    exploration_rate | 0.01     |
| time/               |          |
|    episodes         | 18064    |
|    fps              | 148      |
|    time_elapsed     | 1332     |
|    total_timesteps  | 197298   |
| train/              |          |
|    learning_rate    | 5e-05    |
|    loss             | 2.28     |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 300      |
|    ep_rew_mean      | 2.05     |
|    exploration_rate | 0.01     |
| time/               |          |
|    episodes         | 18124    |
|    fps              | 147      |
|    time_elapsed     | 1338     |
|    total_timesteps  | 198066   |
| train/              |          |
|    learning_rate    | 5e-05    |
|    loss             | 2.37     |
|    n_updates        | 6169     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 300      |
|    ep_rew_mean      | 2.05     |
|    exploration_rate | 0.01     |
| time/               |          |
|    episodes         | 18128    |
|    fps              | 147      |
|    time_elapsed     | 1338     |
|    total_timesteps  | 198108   |
| train/              |          |
|    learning_rate    | 5e-05    |
|    loss             | 3.29     |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 300      |
|    ep_rew_mean      | 2.1      |
|    exploration_rate | 0.01     |
| time/               |          |
|    episodes         | 18188    |
|    fps              | 147      |
|    time_elapsed     | 1343     |
|    total_timesteps  | 198744   |
| train/              |          |
|    learning_rate    | 5e-05    |
|    loss             | 3.68     |
|    n_updates        | 6197     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 300      |
|    ep_rew_mean      | 2.1      |
|    exploration_rate | 0.01     |
| time/               |          |
|    episodes         | 18192    |
|    fps              | 147      |
|    time_elapsed     | 1344     |
|    total_timesteps  | 198786   |
| train/              |          |
|    learning_rate    | 5e-05    |
|    loss             | 2.25     |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 297      |
|    ep_rew_mean      | 2.03     |
|    exploration_rate | 0.01     |
| time/               |          |
|    episodes         | 18252    |
|    fps              | 147      |
|    time_elapsed     | 1349     |
|    total_timesteps  | 199446   |
| train/              |          |
|    learning_rate    | 5e-05    |
|    loss             | 3.37     |
|    n_updates        | 6227     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 297      |
|    ep_rew_mean      | 2.03     |
|    exploration_rate | 0.01     |
| time/               |          |
|    episodes         | 18256    |
|    fps              | 147      |
|    time_elapsed     | 1350     |
|    total_timesteps  | 199494   |
| train/              |          |
|    learning_rate    | 5e-05    |
|    loss             | 2.87     |
|    n_updates      